In [5]:
import pandas as pd

from util.mappings import columns_mapping, incomplete_columns_types

In [ ]:
import glob

files = sorted(glob.glob('data/**/*.csv', recursive=True))

In [ ]:
import pandas as pd

# Read, clean, and store each dataset in a list
final_dfs = []
skipped_files = []
column_names = set()

# def clean_df(df):
    

for file in files:
    try:
        df = pd.read_csv(file, encoding='utf-8')
        df_cleaned = df.dropna()  # Remove null values
        df_renamed = df_cleaned.rename(columns_mapping, axis=1) # Rename columns if necessary
        if "End Station Id" not in df_renamed.columns or "Start Station Id" not in df_renamed.columns:
            print(file)
            continue
        df_corrected = df_renamed.astype(columns_types)
        final_dfs.append(df_corrected)
    except Exception as e:
        skipped_files.append(file)

for file in skipped_files:
    try:
        df = pd.read_csv(file, encoding='cp1252')
        df_cleaned = df.dropna()  # Remove null values
        df_renamed = df_cleaned.rename(columns_mapping, axis=1) # Rename columns if necessary
        if "End Station Id" not in df_renamed.columns or "Start Station Id" not in df_renamed.columns:
            print(file)
            continue
        df_corrected = df_renamed.astype(columns_types)
        final_dfs.append(df_corrected)
    except Exception as e:
        print(file)
        print(e)

# Merge all datasets vertically
merged_df = pd.concat(final_dfs, join='inner', ignore_index=True)


In [ ]:
## extra dfs

leftout_files = ["data/bikeshare-ridership-2017/2017 Data/Bikeshare Ridership (2017 Q3).csv" 
                ,"data/bikeshare-ridership-2017/2017 Data/Bikeshare Ridership (2017 Q4).csv"]

leftout_dfs = []

for file in leftout_files:

    df = pd.read_csv(file, encoding='utf-8')
    df_cleaned = df.dropna()  # Remove null values
    df_renamed = df_cleaned.rename(columns_mapping, axis=1) # Rename columns if necessary
    df_corrected = df_renamed.astype(incomplete_columns_types, errors='ignore')
    leftout_dfs.append(df_corrected)


In [7]:
leftout_merged_df = pd.concat(leftout_dfs, join='inner', ignore_index=True)

In [12]:
leftout_merged_df

,Trip Id,Start Time,End Time,Trip Duration,Start Station Name,End Station Name,User Type
0,1253914,2017-07-01 00:00:00,2017-07-01 00:15:00,910,Princess St / Adelaide St E,424 Wellington St W,Member
1,1253915,2017-07-01 00:01:00,2017-07-01 00:15:00,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
2,1253916,2017-07-01 00:01:00,2017-07-01 00:14:00,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
3,1253917,2017-07-01 00:01:00,2017-07-01 00:25:00,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
4,1253918,2017-07-01 00:01:00,2017-07-01 00:25:00,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
...,...,...,...,...,...,...,...
1026887,2383642,2017-12-31 23:46:27,2017-12-31 23:46:53,26,Bloor St / Brunswick Ave,Bloor St / Brunswick Ave,Casual
1026888,2383643,2017-12-31 23:47:13,2018-01-01 00:11:40,1467,Bloor St / Brunswick Ave,HTO Park (Queens Quay W),Casual
1026889,2383644,2017-12-31 23:47:40,2017-12-31 23:57:49,609,Kendal Ave / Spadina Rd,Augusta Ave / Denison Sq,Member
1026890,2383645,2017-12-31 23:49:08,2017-12-31 23:49:34,26,Phoebe St / Spadina Ave,Phoebe St / Spadina Ave,Member


In [9]:
leftout_stations = set(leftout_merged_df['Start Station Name'].unique()).union(set(leftout_merged_df['End Station Name'].unique()))

In [25]:
other_stations = set(merged_df['Start Station Name'].unique()).union(set(merged_df['End Station Name'].unique()))

In [28]:
import numpy as np

remaining_stations = np.array([*(leftout_stations - other_stations)])
other_stations = np.array([*other_stations])

In [29]:
from Threshold.similarity import similar
import numpy as np

matched = []

similarity_matrix = np.zeros((len(remaining_stations), len(other_stations)))

for i in range(len(remaining_stations)):
    for j in range(len(other_stations)):
        similarity_matrix[i, j] = similar(remaining_stations[i], other_stations[j])

In [30]:
inds = np.argpartition(similarity_matrix, -4, axis=1)[:,-4:]

In [31]:
for i in range(len(remaining_stations)):
    print(remaining_stations[i], list(zip(other_stations[inds[i]], similarity_matrix[i, inds[i]])))
    print("*"*10)

Victoria St / Gould St (Ryerson University) [('King St W / Bay St (West Side)', 0.4931506849315068), ('Bay St / Bloor St W (West Side)', 0.5135135135135135), ('York St / Harbour St (Love Park)', 0.5333333333333333), ('Victoria St / Queen St E', 0.5671641791044776)]
**********
King / Cowan Ave - SMART [('Barton Ave / Howland Ave - SMART', 0.7142857142857143), ('Yonge St / Woburn Ave - SMART', 0.7169811320754716), ('King St W / Cowan Ave - SMART', 0.9056603773584906), ('King St W / Jameson Ave - SMART', 0.7636363636363637)]
**********
Navy Wharf Ct. / Bremner Blvd. [("Strachan Ave / Princes' Blvd", 0.4827586206896552), ('Lower Simcoe St / Bremner Blvd', 0.6), ('Jane St / Yewtree Blvd', 0.5384615384615384), ('Navy Wharf Crt / Bremner Blvd', 0.9491525423728814)]
**********
Summerhill Station [('Temperance St Station', 0.5641025641025641), ('Union Station', 0.5806451612903226), ('Summerhill Subway Station', 0.8372093023255814), ('Rouge Hill GO Station', 0.6666666666666666)]
**********
Lakes

In [18]:
name_changes = {"Dovercourt Rd / Harrison St - SMART": 'Dovercourt Rd / Harrison St (Green P) - SMART',
 "Lake Shore Blvd W / Ontario Dr(Ontario Place)": 'Lake Shore Blvd W / Ontario Dr',
 "Lansdowne Subway Green P": 'Landsdowne Subway Green P',
 "Margueretta St / College St": 'Margueretta St / College St W',
 "Summerhill Ave / MacLennan Ave - SMART": 'Summerhill Ave / Maclennan Ave',
 "Roxton Rd / College St": 'Ossington Ave / College St',
 "Michael Sweet Ave / St. Patrick St": 'Dundas St W / St. Patrick St',
 "Beverly St / College St": 'Beverley St / College St'}

In [19]:
#removal of 'Base Station'

leftout_merged_df = leftout_merged_df[(leftout_merged_df["Start Station Name"] != "Base Station") & (leftout_merged_df["End Station Name"] != "Base Station")]

In [20]:
for change in name_changes.items():

    leftout_merged_df.loc[leftout_merged_df['Start Station Name'] == change[0], 'Start Station Name'] = change[1]

    leftout_merged_df.loc[leftout_merged_df['End Station Name'] == change[0], 'End Station Name'] = change[1]

In [21]:
leftout_stations = set(leftout_merged_df['Start Station Name'].unique()).union(set(leftout_merged_df['End Station Name'].unique()))
other_stations = set(merged_df['Start Station Name'].unique()).union(set(merged_df['End Station Name'].unique()))

remaining_stations = np.array([*(leftout_stations - other_stations)])
other_stations = np.array([*other_stations])

# replacing Fringe Next Stage - 7219 with King St / Victoria St
remaining_stations[remaining_stations == "Fringe Next Stage - 7219"] = "King St / Victoria St"

In [22]:
remaining_stations

array(['Victoria St / Gould St (Ryerson University)',
       'King / Cowan Ave - SMART', 'Navy Wharf Ct. / Bremner Blvd.',
       'Summerhill Station', 'Lakeshore Blvd W / Ellis Ave',
       'Queens Park / Bloor St W', 'York St / King St W - SMART',
       'Simcoe St / Dundas St W', 'Chester Station',
       'Seaton St / Dundas St E',
       'Sorauren/Geoffrey (Sorauren Park) - SMART',
       'St. George St / Russell St - SMART', 'Dufferin St / Bloor St',
       'Bridgeman Ave / Kendal Ave', 'Ward Ave / Wallace Ave',
       'Sumach St / Carlton St. (Riverdale Farm)',
       'King St / Victoria St', 'Bridgeman Ave / Bathurst St',
       'Bloor St / Brunswick Ave', 'Castle Frank Station',
       'Ontario Place Blvd / Remembrance Dr',
       'Bremner Blvd / Spadina Ave', 'Essex St / Christie St',
       'Lower Spadina Ave / Lake Shore Blvd',
       "Wellesley St / Queen's Park Cres", 'Bathurst St / Queens Quay W',
       'Dovercourt Rd / Harrison St (Green P) - SMART',
       'Bay St / Bl

In [ ]:
from Threshold.similarity import similar
import numpy as np

matched = []

similarity_matrix = np.zeros((len(remaining_stations), len(other_stations)))

for i in range(len(remaining_stations)):
    for j in range(len(other_stations)):
        similarity_matrix[i, j] = similar(remaining_stations[i], other_stations[j])

inds = np.argpartition(similarity_matrix, -4, axis=1)[:,-4:]

In [ ]:
for i in range(len(remaining_stations)):
    print(remaining_stations[i], list(zip(other_stations[inds[i]], similarity_matrix[i, inds[i]])))
    print("*"*10)

In [ ]:
new_name_changes = {"Fringe Next Stage - 7219": 'King St E / Victoria St'}

for change in new_name_changes.items():

    leftout_merged_df.loc[leftout_merged_df['Start Station Name'] == change[0], 'Start Station Name'] = change[1]

    leftout_merged_df.loc[leftout_merged_df['End Station Name'] == change[0], 'End Station Name'] = change[1]

In [ ]:
leftout_stations = set(leftout_merged_df['Start Station Name'].unique()).union(set(leftout_merged_df['End Station Name'].unique()))
other_stations = set(merged_df['Start Station Name'].unique()).union(set(merged_df['End Station Name'].unique()))

remaining_stations = np.array([*(leftout_stations - other_stations)])
remaining_stations

In [ ]:
temp1_df = merged_df[["Start Station Id", "Start Station Name"]].rename({'Start Station Id':'Id', 'Start Station Name': 'Name'}, axis=1)
temp2_df = merged_df[["End Station Id", "End Station Name"]].rename({'End Station Id':'Id', 'End Station Name': 'Name'}, axis=1)
temp_df = pd.concat([temp1_df, temp2_df]).drop_duplicates()
temp_df = temp_df.drop_duplicates(subset='Name', keep="last") # this is important to avoid adding multiple ids for a single name
temp_df

In [ ]:
leftout_merged_df = leftout_merged_df.reset_index(drop=True)

In [ ]:
leftout_merged_df['Start Station Id'] = pd.merge(temp_df, leftout_merged_df, left_on='Name', right_on='Start Station Name', how='right')['Id'].astype('int32')
leftout_merged_df['End Station Id'] = pd.merge(temp_df, leftout_merged_df, left_on='Name', right_on='End Station Name', how='right')['Id'].astype('int32')

In [ ]:
len(leftout_merged_df)

In [ ]:
leftout_merged_df.to_csv('2017_Q3_Q4_fixed.csv')